# 一、函数进阶

In [1]:
def f(x = []):
    x.append(5)
    return x
f()

[5]

In [2]:
f()  #直觉上应该返回列表[5],由于默认参数始终使用同一个列表，每次调用默认参数时，函数中的append方法改变了这个列表，因此这个列表会随之改变

[5, 5]

## 1.高阶函数

### 1.函数的对象性

In [2]:
##函数可以作为一种基本类型的对象
##所以函数可以作为一个参数传给另一个参数；将函数作为字典的值；将函数作为另一个函数的返回值
def square(x):
    return(x**2)
d = {"power5": square, "power6": 5}  #将函数值作为字典的值
d["power5"]

<function __main__.square(x)>

In [3]:
d["power5"](5)

25

### 2.以函数为参数的函数（高阶函数）

In [5]:
def apply_function(f, sq):
    return(f(x) for x in sq)
list(apply_function(square, range(5)))  # 将一个函数应用到一个序列的每一个元素中

[0, 1, 4, 9, 16]

In [12]:
tuple(map(square, range(5)))  # python自带的map函数就可以实现这个功能

(0, 1, 4, 9, 16)

### 3.以函数为返回值的函数

In [13]:
def power_func(num):
    def func(x):
        return (x ** num)
    return func
#函数中套函数的子函数，返回一个函数
square2 = power_func(2)
cube2 = power_func(3)
print(square2(5), cube2(5))  #很方便的定义次幂函数

25 125


### 4.固定部分参数的函数（非常好用，py自带）

In [1]:
#python中有一个functools模块中的一个partial（）函数，它接收一个函数作为参数，然后返回一个将这个函数一部分参数固定的新函数
from functools import partial
def power(x, num):
    return (x ** num)
square3 = partial(power, num=2)
square3(4)  #固定次幂参数为2，及此时函数变为了求平方；

16

## 2.函数map（）、filter（）、reduce（）

In [2]:
##map函数之前已经介绍过了，现在介绍另外一个高级函数：filter()
#顾名思义，作用就是筛选,filter(f, sq)作用相当于[x for x in sq if f(x)]
def is_even(x):
    return (x % 2 == 0)
list(filter(is_even, range(6)))

[0, 2, 4]

In [3]:
##两个高级函数还可以叠加使用，效果还不错哦
list(map(square3, list(filter(is_even, range(6)))))

[0, 4, 16]

In [4]:
##另一个好用的高级函数是reduce（），与前两个函数不同的是，reduce函数接收的函数必须是允许二元操作的函数；
##可以用来很快的求解序列内的元素和或者阶乘
from functools import reduce  #调用函数
def multiple(x, y):
    return (x * y)
reduce(multiple, range(1, 5), 10)  #实现阶乘 ,第三个参数是初始值

240

In [5]:
reduce(lambda x,y: x*10 + y, range(1, 6))  #如何把序列拼成一个数

12345

In [6]:
"""
有更好的方法时，最好不要选择reduce函数，因为当应用函数复杂的时候，代码会变的晦涩难懂
比如下面这段代码，把一些科学家的名字按照姓名进行分组
"""
from functools import reduce
scientists =({'name':'Alan Turing', 'age':105, 'gender':'male'},
             {'name':'Dennis Ritchie', 'age':76, 'gender':'male'},
             {'name':'Ada Lovelace', 'age':202, 'gender':'female'},
             {'name':'Frances E. Allen', 'age':84, 'gender':'female'})
def group_by_gender(accumulator , value):
    accumulator[value['gender']].append(value['name'])
    return accumulator
grouped = reduce(group_by_gender, scientists, {'male':[], 'female':[]})
print(grouped)

{'male': ['Alan Turing', 'Dennis Ritchie'], 'female': ['Ada Lovelace', 'Frances E. Allen']}


## 3.lambda()表达式

In [7]:
f = (lambda x: x ** 2)
list(map(f, range(5)))

[0, 1, 4, 9, 16]

## 4.关键字global

In [8]:
x = 5
def foo():
    global x
    x = [1, 2, 3]
    print(x)
foo()  
print(x)  #函数中定义global变量，就可以通过函数改变变量的值

[1, 2, 3]
[1, 2, 3]


In [9]:
x = 5
def foo():
    x = [1, 2, 3]
    print(x)
foo() 
print(x)

[1, 2, 3]
5


## 5.函数的递归

In [29]:
def fib1(n):
    return 1 if n<=1 else fib1(n-1) + fib1(n-2)
list(map(fib1, range(10)))  #普通的递归

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [30]:
def fib2(n):
    a, b = 1, 1
    for _ in range(n):
        a, b = b, a+b
    return a
list(map(fib2, range(10)))  #非递归的方法

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [31]:
def fib3(n, cache={0:1, 1:1}):
    try:
        return cache[n]
    except KeyError:
        cache[n] = fib3(n-1) + fib3(n-2)
        return cache[n]

list(map(fib3, range(10)))  #使用默认缓存对递归进行优化

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [32]:
%timeit fib1(30)

432 ms ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%timeit fib2(30)

2.62 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [65]:
%timeit fib3(30)

155 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# 二、迭代器与生成器

## 1.迭代器(Iterator)

In [4]:
##容器类型通常包含一个迭代器，帮助它们支持for循环的操作
##要实现一个._iter_()方法返回相应的迭代器
x = (1, 2, 3)
i = x.__iter__()

In [5]:
print(next(i))
print(next(i))
print(next(i))

1
2
3


In [6]:
next(i)  #当迭代完成后，迭代器就会报错

StopIteration: 

In [7]:
i.__iter__() is i  # 迭代器的特点之一是：迭代器的迭代器还是其本身

True

In [22]:
import collections  #为了查看定义的对象是否可迭代
"""
自定义数字自加迭代器，加到20为止
对于一个迭代器来说，定义是：__iter__()方法返回迭代器本身，__next__()方法进行迭代，知道出错为止
"""
class MyNumbers:
  def __iter__(self):
    self.a = 1
    return self
 
  def __next__(self):
    if self.a <= 20:        
        x = self.a
        self.a += 1
        return x
    else:
        raise StopIteration  
myclass = MyNumbers()
print("是否为可迭代对象：%s"%isinstance(myclass,collections.Iterable))
myclass.__iter__()
myclass.__next__()  #自定义加数迭代器
for i in myclass:
    print(i,)
  #迭代器对象可以作为for循环的对象，将内容依次输出

是否为可迭代对象：True
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [20]:
##自定义逆序迭代器
import collections  #为了查看定义的对象是否可迭代
class Reverse:
    def __init__(self, x):
        self.seq = x
        self.idx = len(x)

    def __iter__(self):
        return self

    def __next__(self):
        self.idx -= 1
        if self.idx >= 0:
            return self.seq[self.idx]
        else:
            raise StopIteration

x = list(range(10))
print("是否为可迭代对象：%s"%isinstance(Reverse(x),collections.Iterable))
type(Reverse(x))
Reverse(x).__iter__()
for i in Reverse(x):
    print(i, end=",")  #end参数可以调节输入之间的间隔符号

是否为可迭代对象：True
9,8,7,6,5,4,3,2,1,0,

In [32]:
"""
Collatz猜想
有一位数学家猜想，从任意的正整数n开始使用一定的规则迭代，总能在有限次操作内使n为1
自定义迭代器对象，功能是，对于任意输入整数进行迭代，直到数字变为1
"""
class Collatz(object):
    def __init__(self, n):
        self.value = n
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.value == 1:
            raise StopIteration
        elif self.value % 2 == 0:
            self.value = self.value / 2
        else:
            self.value = 3 * self.value + 1
        return self.value
for i in Collatz(7):
    print (i, end=" ")


22 11.0 34.0 17.0 52.0 26.0 13.0 40.0 20.0 10.0 5.0 16.0 8.0 4.0 2.0 1.0 

## 2.生成器

#### 使用类实现自定义类型的迭代器比较麻烦，所以有一种用函数生成的迭代器更为方便，那就是生成器

In [2]:
"""
还是对于Collatzcai猜想，使用生成器生成,注意生成器中的返回不是return，是yield
"""

def collatz(n):
    while n != 1:
        if n % 2 == 0:
            n /= 2
        else:
            n = 3*n + 1
        yield n

for x in collatz(7):
    print(x, end=" ")


22 11.0 34.0 17.0 52.0 26.0 13.0 40.0 20.0 10.0 5.0 16.0 8.0 4.0 2.0 1.0 

In [29]:
g = collatz(7)
print(g.__next__())
print(g.__iter__() is g)  #经过验证，生成器就是一种特殊的迭代器
g

22
True


<generator object collatz at 0x0000029A029AB8B8>

In [21]:
"""
逆序数的生成器实现
"""
def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]

for c in reverse("abcd"):
    print(c, end=" ")


d c b a 

In [23]:
(x for x in range(10))  可见，py中利用for循环生成列表的操作是生成器

<generator object <genexpr> at 0x0000029A029AB7C8>

In [31]:
"""
使用生成器或者迭代器，不需要一次性保存序列的所有值，可以在需要的时候计算，从而节省内存空间
"""

'\n使用生成器或者迭代器，不需要一次性保存序列的所有值，可以在需要的时候计算，从而节省内存空间\n'

# 三、装饰器(接受函数作为参数的函数)

In [3]:
def foo(x):
    print(x)

foo  #py中，函数本身其实也是个对象

<function __main__.foo(x)>

In [36]:
contents = dir(foo)  #使用dir()函数查看对象的属性和方法

In [26]:
foo.__call__(25)  #使用foo(x)相当于调用了__call__()方法

25


In [4]:
"""
定义一个函数，输入一个函数，打印出函数的名字
"""
def nameit(f):
    print("calling function", f.__name__)
    return f
nameit(foo)

calling function foo


<function __main__.foo(x)>

In [11]:
##这里定义装饰器时，有一些小小的瑕疵，就是在没有调用函数时，还是有了函数的信息；这时我们可以使用函数中套函数的方法进行完善
##如下所示：
def nameit_b(f):
    def g(*args, **kwargs):
        print("calling function", f.__name__)
        return f(*args, **kwargs)
    return g

nameit_b(foo)(3)

calling function foo
3


对装饰器的改进

In [6]:
nameit(foo)(25)  #比起上面的结果，现在的结果中，多了一行名称，，而这行文字是通过函数来定义实现的，这个给函数加特技的函数就叫装饰器

calling function foo
25


In [12]:
@nameit_b
def add(x, y):
    return (x+y)  #想使用装饰器，只需在定义函数的时候在前面加@即可，十分的方便，而且装饰器可以使用多个

add(3,5)

calling function add


8

In [14]:
assert (isinstance(5.0, int))  #assert关键字，作为py中的关键字，这个可以用来检测之后的表达式是否为真，如果为假程序就会报错

AssertionError: 

多个装饰器是可以叠加使用的

In [17]:
##多个装饰器一起使用
def plus_one(f):
    def new_func(x):
        return (f(x) + 1)
    return new_func

def mult_two(f):
    def new_func(x):
        return (f(x) * 2)
    return new_func

@nameit_b  #A
@plus_one  #B
@mult_two  #C
def foo(x):
    return(x)  #f=A(B(C(f)))

foo(5)

calling function new_func


11

装饰器还可以进行加工化，成为一个装饰器工厂（批量化产生装饰器）

In [26]:
##对于上面的装饰器可以定义f（x）+n的装饰器，以及f（x）*n的装饰器

def plus_n(n):
    def plus_dec(f):
        def new_func(x):
            return (f(x) + n)  #返回加法运算结果
        return new_func  #根据f返回装饰后的函数
    return plus_dec  #根据n返回特定加法装饰器
#加法装饰器

def mult_n(n):
    def mult_dec(f):
        def new_func(x):
            return (f(x) * n)
        return new_func
    return mult_dec
##乘法装饰器

@nameit_b  #A
@plus_n(5)  #B
@mult_n(6)  #C
def foo(x):
    return(x)  #f=A(B(C(f)))

foo(5)

calling function new_func


35

# 四、上下文管理器与with语句

py提供了上下文管理器的机制来解决这个问题，通常通过with使用
它是一个实现了__enter__()方法和__exit__()方法的对象

In [1]:
f = open("test.txt", "w")
f.__enter__

<function TextIOWrapper.__enter__>

In [2]:
f.__exit__

<function TextIOWrapper.__exit__>

In [3]:
f.close()  

In [4]:
"""
自定义一个上下文管理器对象
"""
class TestManager(object):
    def __enter__(self):
        print("Entering")
    
    def __exit__(self, exc_type, exc_value, tracebcak):  #__exit__方法需要接受3个额外参数
        print("Exiting")
    
with TestManager():
    print("Hello~")  #使用管理器（用with关键字），先调用enter方法，然后语句，然后exit方法



Entering
Hello~
Exiting


In [5]:
with TestManager():
    a = 1/0  #若with中的语句出错，就会调用exit方法


Entering
Exiting


ZeroDivisionError: division by zero

In [6]:
##enter方法返回的值刚好是它本身,即enter方法返回的是一个上下文管理器
f = open("test.txt", "w")
f.__enter__() is f

True

In [7]:
class TestManager(object):
    def __enter__(self):
        return (self)  #返回管理器本身
    
    def __exit__(self, exc_type, exc_value, tracebcak):  #__exit__方法需要接受3个额外参数
        print("Exiting")

with TestManager() as value:  #将TestManager()管理器as为value，本质上就是把enter（）方法的返回值赋值给value
    print(value)


Exiting


## 3.方法exit()与异常处理

注意到上文中，定义exit方法时，需要额外输入三个参数，这三个参数与异常处理有关

In [9]:
class TestManager(object):
    def __enter__(self):
        return (self)  #返回管理器本身
    
    def __exit__(self, exc_type, exc_value, traceback):  #__exit__方法需要接受3个额外参数
        print("Exiting")
        print("Arg:", exc_type)
        print("Arg:", exc_value)
        print("Arg:", traceback)

with TestManager():
    a = 1/0

##可以看到运行异常时，这三个变量记录了错误的相关信息

Exiting
Arg: <class 'ZeroDivisionError'>
Arg: division by zero
Arg: <traceback object at 0x0000022A36D9FE08>


ZeroDivisionError: division by zero

In [10]:
class TestManager(object):
    def __enter__(self):
        return (self)  #返回管理器本身
    
    def __exit__(self, exc_type, exc_value, traceback):  #__exit__方法需要接受3个额外参数
        print("Exiting")
        print("Arg:", exc_type)
        print("Arg:", exc_value)
        print("Arg:", traceback)
        return True
with TestManager():
    a = 1/0

##将exit 的返回值设置为True就可以不显示错误的信息啦

Exiting
Arg: <class 'ZeroDivisionError'>
Arg: division by zero
Arg: <traceback object at 0x0000022A36D9FAC8>


In [11]:
###文件管理器的用处在于，不需要自己编写try和finally代码块增加工作量；通过with可以达到同样的效果
###让文件写入在出错时，能够正常关闭，不会出错

## 4.模块contextlib

py提供了contextlib模块来方便的使用上下文管理器

### 1.contextmanager装饰器

In [12]:
import contextlib
@contextlib.contextmanager
def test_manager():
    print("Hello!")
    yield ("HaHa")
    print("Bye!")

with test_manager() as value:
    print(value)  #看得出这其实是个上下文管理器

Hello!
HaHa
Bye!


着重介绍一下contextmanager装饰器；
能够将一个函数改变成一个上下文管理器；
但是对函数有一些要求：1.必须是一个生成器；2.yield只能被执行一次；3.yield之前的可以看做enter方法的内部；yield返回的就是enter方法的返回值；yield 之后的可以看做exit方法的内部


In [13]:
##但是这种方法有一个问题，就是不能够在主体出错时，执行exit方法，所以需要自己使用try块进行修改
@contextlib.contextmanager
def test_manager():
    print("Hello!")
    try:
        yield ("HaHa")
    except Exception as exc:
        print("Error:", exc)
    finally:
        print("Bye!")

with test_manager():
    a = 1/0




Hello!
Error: division by zero
Bye!


### 2.closing函数

closing()函数，该函数接受一个对象，返回一个确保该对象的.close()方法被调用的上下文管理器

In [14]:
##实例如下
from urllib import request
from contextlib import closing

with closing(request.urlopen("https://www.accaglobal.com/uk/en.html")) as url:
    line = url.readline()
    print(line)


b'\r\n'


In [15]:
a = request.urlopen("https://www.accaglobal.com/uk/en.html")
a.getcode()  #200代表允许访问，连接成功

200

# 五、变量作用域

python中有函数局部变量；全局变量；内置变量作用域；以及闭包作用域

In [16]:
c = 1
def plus(a, b):
    c = 2
    return(a+b+c)

print(c)  #函数内的局部变量不会影响函数外部的全局变量的
plus(1, 2)

1


5

In [17]:
c = 1
def plus(a, b):
    global c
    c = 3
    return(a+b+c)

print(plus(1, 2))
print("c =",c)  #此时就改变了c的值，因为函数内部将c申明为了全局变量

6
c = 3


In [18]:
def outer():
    a = 1
    def inner():
        return a
    return inner

outer()()  #对于inner函数来说，a既不是局部变量也不是全局变量，py中称之为闭包变量作用域

1